### Create File with that matches zcta codes to their pre and post redistricting districts. 
#### This has been done in the calc block dists all us.ipyr, but I'm doing it again here for clarity and to keep track of the file.

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import geoplot
import math
import os
from geopandas.tools import geocode
from geopandas.tools import sjoin
from shapely.geometry import LinearRing
from shapely.geometry import Point
from shapely.ops import nearest_points
          

In [2]:
#Change to data directory where district shapefiles and block shapefiles stored
data="C:/Users/Neil/Dropbox/pork spending"#cd1=cd114_paoh[(cd114_paoh['CD114FP']=='12')  & (cd114_paoh['STATEFP']=='42')].boundary.iloc[0]
#closest_coords(polygon=cd1,point=PA_bgcentroids.iloc[0])
#PA_bg['cp1_x']=
#print(PA_bg.crs)
#cols=['CD1','CD2','CD3','CD4','CD5','CD6','CD7','CD8','CD9','CD10','CD11','CD12','CD13','CD14','CD15','CD16','CD17','CD18']
stfips_map={
    'AL' : ['01','Alabama'],
    'AK' : ['02','Alaska'],
    'AZ' : ['04','Arizona'],
    'AR' : ['05','Arkansas'],
    'CA' : ['06','Calafornia'],
    'CO' : ['08','Colorado'],
    'CT' : ['09','Connecticut'],
    'DE' : ['10','Delaware'],
    'FL' : ['12','Florida'],
    'GA' : ['13','Georgia'],
    'HI' : ['15','Hawaii'],
    'ID' : ['16','Idaho'],
    'IL' : ['17','Illinois'],
    'IN' : ['18','Indiana'],
    'IA' : ['19','Iowa'],
    'KS' : ['20','Kansas'],
    'KY' : ['21','Kentucky'],
    'LA' : ['22','Louisiana'],
    'ME' : ['23','Maine'],
    'MD' : ['24','Maryland'],
    'MA' : ['25','Massachusetts'],
    'MI' : ['26','Michigan'],
    'MN' : ['27','Minnesota'],
    'MS' : ['28','Mississippi'],
    'MO' : ['29','Montana'],
    'MT' : ['30','Montana'],
    'NE' : ['31','Nebraska'],
    'NV' : ['32','Nevada'],
    'NH' : ['33','New Hampshire'],
    'NJ' : ['34','New Jersey'],
    'NM' : ['35','New Mexico'],
    'NY' : ['36','New York'],
    'NC' : ['37','North Carolina'],
    'ND' : ['38','North Dakota'],
    'OH' : ['39','Ohio'],
    'OK' : ['40','Oklahoma'],
    'OR' : ['41','Oregon'],
    'PA' : ['42','Pennsylvania'],
    'RI' : ['44','Rhode Island'],
    'SC' : ['45','South Carolina'],
    'SD' : ['46','South Dakota'],
    'TN' : ['47','Tennessee'],
    'TX' : ['48','Texas'],
    'UT' : ['49','Utah'],
    'VT' : ['50','Vermont'],
    'VA' : ['51','Virginia'],
    'WA' : ['53','Washington'],
    'WV' : ['54','West Virginia'],
    'WI' : ['55','Wisconsin'],
    'WY' : ['56','Wyoming'],
    'AS' : ['60','American Samoa'],
    'GU' : ['66','Guam'],
    'MP' : ['69','Northern Mariana Islands'],
    'PR' : ['72','Puerto Rico'],
    'VI' : ['78','Virgin Islands'],
    'ALL':['','']
}

nonstates = ['AS','GU' ,'MP' ,'PR' ,'VI','ALL' ]

stnamefipsmap={
    "Alabama" : "01",
    "Alaska" : "02",
    "Arizona" : "04",
    "Arkansas" : "05",
    "California" : "06",
    "Colorado" : "08",
    "Connecticut" : "09",
    "Delaware" : "10",
    "Florida" : "12",
    "Georgia" : "13",
    "Hawaii" : "15",
    "Idaho" : "16",
    "Illinois" : "17",
    "Indiana" : "18",
    "Iowa" : "19",
    "Kansas" : "20",
    "Kentucky" : "21",
    "Louisiana" : "22",
    "Maine" : "23",
    "Maryland" : "24",
    "Massachusetts" : "25",
    "Michigan" : "26",
    "Minnesota" : "27",
    "Mississippi" : "28",
    "Missouri" : "29",
    "Montana" : "30",
    "Nebraska" : "31",
    "Nevada" : "32",
    "New Hampshire" : "33",
    "New Jersey" : "34",
    "New Mexico" : "35",
    "New York" : "36",
    "North Carolina" : "37",
    "North Dakota" : "38",
    "Ohio" : "39",
    "Oklahoma" : "40",
    "Oregon" : "41",
    "Pennsylvania" : "42",
    "Rhode Island" : "44",
    "South Carolina" : "45",
    "South Dakota" : "46",
    "Tennessee" : "47",
    "Texas" : "48",
    "Utah" : "49",
    'Vermont' : "50",
    "Virginia" : "51",
    "Washington" : "53",
    "West Virginia" : "54",
    "Wisconsin" : "55",
    "Wyoming" : "56"
              }


In [3]:
stname_fips=pd.DataFrame(list(stnamefipsmap.items()), columns=['stname','STATEFP'])


#### Import district maps

In [4]:
#Import district MAPS
distmapcd105=gpd.read_file(data + "/national congressial district shapefiles/districts105/districtShapes/districts105.shp")
distmapcd105=distmapcd105.merge(stname_fips,left_on='STATENAME',right_on='stname', how='left')
distmapcd105=distmapcd105.rename(columns={'DISTRICT':'cd105','STATEFP':'STATEFP_105'})
distmapcd105=distmapcd105[distmapcd105.geometry.type.isin(['Polygon','MultiPolygon'])]

distmapcd106=gpd.read_file(data + "/national congressial district shapefiles/districts106/districtShapes/districts106.shp")
distmapcd106=distmapcd106.merge(stname_fips,left_on='STATENAME',right_on='stname', how='left')
distmapcd106=distmapcd106.rename(columns={'DISTRICT':'cd106','STATEFP':'STATEFP_106'})
distmapcd106=distmapcd106[distmapcd106.geometry.type.isin(['Polygon','MultiPolygon'])]

distmapcd107=gpd.read_file(data + "/national congressial district shapefiles/districts107/districtShapes/districts107.shp")
distmapcd107=distmapcd107.merge(stname_fips,left_on='STATENAME',right_on='stname', how='left')
distmapcd107=distmapcd107.rename(columns={'DISTRICT':'cd107','STATEFP':'STATEFP_107'})
distmapcd107=distmapcd107[distmapcd107.geometry.type.isin(['Polygon','MultiPolygon'])]


distmapcd108=gpd.read_file(data + "/national congressial district shapefiles/districts108/districtShapes/districts108.shp")
distmapcd108=distmapcd108.merge(stname_fips,left_on='STATENAME',right_on='stname', how='left')
distmapcd108=distmapcd108.rename(columns={'DISTRICT':'cd108','STATEFP':'STATEFP_108'})
distmapcd108=distmapcd108[distmapcd108.geometry.type.isin(['Polygon','MultiPolygon'])]


distmapcd109=gpd.read_file(data + "/national congressial district shapefiles/districts109/districtShapes/districts109.shp")
distmapcd109=distmapcd109.merge(stname_fips,left_on='STATENAME',right_on='stname', how='left')
distmapcd109=distmapcd109.rename(columns={'DISTRICT':'cd109','STATEFP':'STATEFP_109'})
distmapcd109=distmapcd109[distmapcd109.geometry.type.isin(['Polygon','MultiPolygon'])]


distmapcd110=gpd.read_file(data + "/national congressial district shapefiles/districts110/districtShapes/districts110.shp")
distmapcd110=distmapcd110.merge(stname_fips,left_on='STATENAME',right_on='stname', how='left')
distmapcd110=distmapcd110.rename(columns={'DISTRICT':'cd110','STATEFP':'STATEFP_110'})
distmapcd110=distmapcd110[distmapcd110.geometry.type.isin(['Polygon','MultiPolygon'])]


distmapcd111=gpd.read_file(data + "/national congressial district shapefiles/tl_2010_us_cd111/tl_2010_us_cd111.shp")
distmapcd111=distmapcd111.rename(columns={'CD111FP':'cd111','STATEFP10':'STATEFP_111'})
distmapcd112=gpd.read_file(data + "/national congressial district shapefiles/tl_2011_us_cd112/tl_2011_us_cd112.shp")
distmapcd112=distmapcd112.rename(columns={'CD112FP':'cd112','STATEFP':'STATEFP_112'})    
distmapcd113=gpd.read_file(data + "/national congressial district shapefiles/tl_2013_us_cd113/tl_2013_us_cd113.shp")
distmapcd113=distmapcd113.rename(columns={'CD113FP':'cd113','STATEFP':'STATEFP_113'})   
distmapcd114=gpd.read_file(data + "/national congressial district shapefiles/tl_2015_us_cd114/tl_2015_us_cd114.shp")
distmapcd114=distmapcd114.rename(columns={'CD114FP':'cd114','STATEFP':'STATEFP_114'})

districtmap_dict={'cd105':distmapcd105,
                  'cd106':distmapcd106,
                  'cd107':distmapcd107,
                  'cd108':distmapcd108,
                  'cd109':distmapcd109,
                  'cd110':distmapcd110,
                  'cd111':distmapcd111,
                  'cd112':distmapcd112,
                  'cd113':distmapcd113,
                  'cd114':distmapcd114}

#### Import Zip code (zcta) map

In [5]:
#Import zip map
zcta=gpd.read_file(data + "/tl_2019_us_zcta510/tl_2019_us_zcta510.shp")
zcta.loc[:,'centroid']=zcta.geometry.centroid
zcta = zcta.set_geometry('centroid')
zcta = zcta[['ZCTA5CE10','geometry','centroid']]

#### Assign zcta to district based on centroid

In [16]:
# Make sure they're using the same projection reference
districtmap_dict['cd105'].crs = zcta.crs
districtmap_dict['cd106'].crs = zcta.crs
districtmap_dict['cd107'].crs = zcta.crs
districtmap_dict['cd108'].crs = zcta.crs
districtmap_dict['cd109'].crs = zcta.crs
districtmap_dict['cd110'].crs = zcta.crs

#sjoin
zips_with_districts_centroids=sjoin(zcta,districtmap_dict['cd105'][['geometry','STATEFP_105','cd105']],how="left")
zips_with_districts_centroids=zips_with_districts_centroids.drop(columns=['index_right'])

zips_with_districts_centroids=sjoin(zips_with_districts_centroids,districtmap_dict['cd106'][['geometry','STATEFP_106','cd106']],how="left")
zips_with_districts_centroids=zips_with_districts_centroids.drop(columns=['index_right'])

zips_with_districts_centroids=sjoin(zips_with_districts_centroids,districtmap_dict['cd107'][['geometry','STATEFP_107','cd107']],how="left")
zips_with_districts_centroids=zips_with_districts_centroids.drop(columns=['index_right'])

zips_with_districts_centroids=sjoin(zips_with_districts_centroids,districtmap_dict['cd108'][['geometry','STATEFP_108','cd108']],how="left")
zips_with_districts_centroids=zips_with_districts_centroids.drop(columns=['index_right'])

zips_with_districts_centroids=sjoin(zips_with_districts_centroids,districtmap_dict['cd109'][['geometry','STATEFP_109','cd109']],how="left")
zips_with_districts_centroids=zips_with_districts_centroids.drop(columns=['index_right'])

zips_with_districts_centroids=sjoin(zips_with_districts_centroids,districtmap_dict['cd110'][['geometry','STATEFP_110','cd110']],how="left")
zips_with_districts_centroids=zips_with_districts_centroids.drop(columns=['index_right'])
# Make sure they're using the same projection reference
districtmap_dict['cd111'].crs = zcta.crs
#sjoin
zips_with_districts_centroids=sjoin(zips_with_districts_centroids,districtmap_dict['cd111'][['geometry','STATEFP_111','cd111']],how="left")
zips_with_districts_centroids=zips_with_districts_centroids.drop(columns=['index_right'])

zips_with_districts_centroids=sjoin(zips_with_districts_centroids,districtmap_dict['cd112'][['geometry','STATEFP_112','cd112']],how="left")
zips_with_districts_centroids=zips_with_districts_centroids.drop(columns=['index_right'])

zips_with_districts_centroids=sjoin(zips_with_districts_centroids,districtmap_dict['cd113'][['geometry','STATEFP_113','cd113']],how="left")
zips_with_districts_centroids=zips_with_districts_centroids.drop(columns=['index_right'])

zips_with_districts_centroids=sjoin(zips_with_districts_centroids,districtmap_dict['cd114'][['geometry','STATEFP_114','cd114']],how="left")
zips_with_districts_centroids=zips_with_districts_centroids.drop(columns=['index_right'])

outfile=pd.DataFrame(zips_with_districts_centroids.drop(columns=['geometry','centroid']))
outfile=outfile.rename(columns={'ZCTA5CE10':'zip_code'})

outfile['zip_code']=outfile.zip_code.astype(int)
outfile.to_stata(data + "/zips_with_districts_centroids.dta")

#### Assign zcta to district based on intersection

In [ ]:
# Make sure they're using the same projection reference
districtmap_dict['cd111'].crs = zcta.crs
# Change zip geometry to polygon
zcta = zcta.set_geometry('geometry')
#sjoin
zips_with_districts_intersect=sjoin(zcta,districtmap_dict['cd105'][['geometry','STATEFP_105','cd105']],how="left")
zips_with_districts_intersect=sjoin(zcta,districtmap_dict['cd106'][['geometry','STATEFP_106','cd106']],how="left")
zips_with_districts_intersect=sjoin(zcta,districtmap_dict['cd107'][['geometry','STATEFP_107','cd107']],how="left")
zips_with_districts_intersect=sjoin(zcta,districtmap_dict['cd108'][['geometry','STATEFP_108','cd108']],how="left")
zips_with_districts_intersect=sjoin(zcta,districtmap_dict['cd109'][['geometry','STATEFP_109','cd109']],how="left")
zips_with_districts_intersect=sjoin(zcta,districtmap_dict['cd110'][['geometry','STATEFP_110','cd110']],how="left")

zips_with_districts_intersect=sjoin(zcta,districtmap_dict['cd111'][['geometry','STATEFP_111','cd111']],how="left")
zips_with_districts_intersect=zips_with_districts_intersect.drop(columns=['index_right'])
zips_with_districts_intersect=sjoin(zips_with_districts_intersect,districtmap_dict['cd112'][['geometry','STATEFP_112','cd112']],how="left")
zips_with_districts_intersect=zips_with_districts_intersect.drop(columns=['index_right'])
zips_with_districts_intersect=sjoin(zips_with_districts_intersect,districtmap_dict['cd113'][['geometry','STATEFP_113','cd113']],how="left")
zips_with_districts_intersect=zips_with_districts_intersect.drop(columns=['index_right'])
zips_with_districts_intersect=sjoin(zips_with_districts_intersect,districtmap_dict['cd114'][['geometry','STATEFP_114','cd114']],how="left")
zips_with_districts_intersect=zips_with_districts_intersect.drop(columns=['index_right'])